In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import pandasql
from pandasql import sqldf
import plotly.express as px

pysqldf = lambda q: sqldf(q, globals())

In [12]:
circuits = pd.read_csv('../data/circuits.csv')
constructor_results = pd.read_csv('../data/constructor_results.csv')
constructor_standings = pd.read_csv('../data/constructor_standings.csv')
constructor = pd.read_csv('../data/constructors.csv')
drivers = pd.read_csv('../data/drivers.csv')
driver_standings = pd.read_csv('../data/driver_standings.csv')
lap_times = pd.read_csv('../data/lap_times.csv')
pit_stops = pd.read_csv('../data/pit_stops.csv')
qualif = pd.read_csv('../data/qualifying.csv')
races = pd.read_csv('../data/races.csv')
results = pd.read_csv('../data/results.csv')
seasons = pd.read_csv('../data/seasons.csv')
status = pd.read_csv('../data/status.csv')

# Data Cleaning

We remove the url links from the datasets and combine some of the datasets for easier readability.

In [13]:
circuits = circuits.drop("url", axis=1)
constructor = constructor.drop("url", axis=1)
drivers = drivers.drop("url", axis=1)
seasons = seasons.drop("url", axis=1)
races = races.drop("url", axis=1)

# Data Exploration

- constructor wins by circuit

In [14]:
q = """select 
C.raceId, 
C.constructorId,
CName.name,
C.points,
C.position, 
C.wins,
R.circuitID,
R.name as circuitName
from constructor_standings C
inner join races R
on C.raceID = R.raceID
inner join constructor CName
on CName.constructorId = C.constructorId
where C.position = 1
or C.position = 2
or C.position = 3
;"""
constructor_standings_cleaned = pysqldf(q)
constructor_standings_cleaned

,raceId,constructorId,name,points,position,wins,circuitId,circuitName
0,18,1,McLaren,14.0,1,1,1,Australian Grand Prix
1,18,2,BMW Sauber,8.0,3,0,1,Australian Grand Prix
2,18,3,Williams,9.0,2,0,1,Australian Grand Prix
3,19,1,McLaren,24.0,1,1,2,Malaysian Grand Prix
4,19,2,BMW Sauber,19.0,2,0,2,Malaysian Grand Prix
...,...,...,...,...,...,...,...,...
2923,1054,9,Red Bull,83.0,2,1,75,Portuguese Grand Prix
2924,1054,1,McLaren,53.0,3,0,75,Portuguese Grand Prix
2925,1055,131,Mercedes,141.0,1,3,4,Spanish Grand Prix
2926,1055,9,Red Bull,112.0,2,1,4,Spanish Grand Prix


In [15]:
def get_win_location_count(constructor_standings_cleaned):
    win_locations = {}
    for row in constructor_standings_cleaned.iterrows():
        constructorID = row[1]['name']
        circuitName = row[1]['circuitName']
        if constructorID not in win_locations:
            win_locations[constructorID] = {}
            win_locations[constructorID][circuitName]= 1
        else:
            if circuitName not in win_locations[constructorID]:
                win_locations[constructorID][circuitName]= 1
            else:
                win_locations[constructorID][circuitName] += 1
    location_df = pd.DataFrame(win_locations)
    return win_locations, location_df

win_locations, location_df = get_win_location_count(constructor_standings_cleaned)
location_df.reset_index(inplace=True)

Questions to answer here:
- Is there a particular constructor that dominates a circuit and has most wins?
- Is there a circuit that a constructor is relatively better at?

In [16]:
location_na = location_df.T.isna().sum()
for k, circuit in enumerate(list(location_na)):
    if circuit > 38:
        location_df.drop(k, axis=0, inplace=True)

constructor_na = location_df.isna().sum()
drop_list = []
for k, constructor_n in enumerate(list(constructor_na)):
    if constructor_n > 20:
        drop_list.append(k)
location_df.drop(location_df.columns[drop_list], axis=1, inplace=True)

How big of an effect do pit stops have on winning the race?

In [37]:
# Let's start with the 2020 season:

q = """select 
P.raceId,
D.driverRef,
P.stop,
P.lap,
P.milliseconds,
DS.position
from pit_stops P
left join drivers D
on P.driverId = D.driverId
left join driver_standings DS
on P.raceId = DS.raceId
and P.driverId = DS.driverId
where P.raceId > 1030
and P.raceId < 1048
;"""
pit_stop_corr = pysqldf(q)

In [38]:
pit_stop_corr = pit_stop_corr[pit_stop_corr['milliseconds'] < 60000]

In [39]:
df = px.data.iris()
# fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
#                     color='petal_length', symbol='species')
# fig.show()

fig = px.scatter_3d(pit_stop_corr, x='milliseconds', y='raceId', z='position',
                    color='driverRef')
fig.show()

,raceId,year,round,circuitId,name,date,time
1018,1031,2020,1,70,Austrian Grand Prix,2020-07-05,13:10:00
1019,1032,2020,2,70,Styrian Grand Prix,2020-07-12,13:10:00
1020,1033,2020,3,11,Hungarian Grand Prix,2020-07-19,13:10:00
1021,1034,2020,4,9,British Grand Prix,2020-08-02,13:10:00
1022,1035,2020,5,9,70th Anniversary Grand Prix,2020-08-09,13:10:00
1023,1036,2020,6,4,Spanish Grand Prix,2020-08-16,13:10:00
1024,1037,2020,7,13,Belgian Grand Prix,2020-08-30,13:10:00
1025,1038,2020,8,14,Italian Grand Prix,2020-09-06,13:10:00
1026,1039,2020,9,76,Tuscan Grand Prix,2020-09-13,13:10:00
1027,1040,2020,10,71,Russian Grand Prix,2020-09-27,11:10:00


In [20]:
pit_stops

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842
...,...,...,...,...,...,...,...
8218,1055,822,2,53,16:18:52,21.638,21638
8219,1055,815,2,57,16:24:55,21.345,21345
8220,1055,844,2,58,16:26:05,22.292,22292
8221,1055,830,2,60,16:28:18,21.592,21592


In [ ]:
# Pit-Stop Corr
# Lap-Time Correlation
# Lap Time changes over the years and correlation with FIA regulations -- FIA ban on Williams cars 1990s
# Which seasons did drivers/constructors completely dominate?
# 1 stop vs 2 stop strategy for circuits